In [69]:
import boto3
import os
from dotenv import load_dotenv
from smart_open import smart_open
from typing import Optional
import pandas as pd
import numpy as np

class S3AWS:
    """
    S3AWS represents aws object storage s3. It handles necessary features including
    authenticating to access to s3, and creating bucket to store the ingested data
    """
    def __init__(self, access_key_id: Optional[str], secret_key_id: Optional[str]):
        self.access_key_id = access_key_id
        self.secret_key_id = secret_key_id

    def create_bucket(self, bucket_name: str):
        """
        Create S3 bucket with bucket_name
        """
        client = boto3.client('s3', 
                           aws_access_key_id=self.access_key_id,
                           aws_secret_access_key=self.secret_key_id)
        client.create_bucket(Bucket=bucket_name)

        return client

    def load_df(self, bucket_name: str, key: str, type: str, sheet: int) -> pd.DataFrame:
        path = f"s3://{self.access_key_id}:{self.secret_key_id}@{bucket_name}/{key}"
        if type == "csv":
            return pd.read_csv(smart_open(path))
        elif type == "xls":
            return pd.read_excel(smart_open(path), sheet)
        
    def df_to_s3(self, dataframe: pd.DataFrame, bucket_name: str, key: str, ) -> None:
        s3_bucket = self.create_bucket(bucket_name)
        if s3_bucket:
            s3_bucket.put_object(Bucket=bucket_name, Body=dataframe.to_csv(None).encode(), Key=key)

In [70]:
load_dotenv()
s3 = S3AWS(os.getenv("ACCESS_KEY_ID"),os.getenv("SECRET_ACCESS_KEY"))
df_raw = s3.load_df("s3-bucket-raw-usda", "loss-adjusted-food-availability/calories.xls", "xls", 1)
df_raw.head()

,"Average daily per capita calories from the U.S. food availability, adjusted for spoilage and other waste",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Year,"Meat, eggs, and nuts",Dairy,Fruit,Vegetables,Flour and cereal products**,Added fats and oils and dairy fats*,Sugar and sweeteners (Added),Total
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970,508.895424,250.291001,71.242053,135.44114,409.802437,346.18452,332.539441,2054.396014


In [71]:
df = df_raw.copy()
df.columns = df.iloc[0]
last_index = df[df['Year'] == 2017].index[0] + 1
df = df[4: last_index]
df = df.set_index("Year")
fod_median = df["Added fats and oils and dairy fats*"].median()
total_median = df["Total"].median()
df["Added fats and oils and dairy fats*"] = df["Added fats and oils and dairy fats*"].fillna(fod_median)
df["Total"] = df["Total"].fillna(total_median)
df

,"Meat, eggs, and nuts",Dairy,Fruit,Vegetables,Flour and cereal products**,Added fats and oils and dairy fats*,Sugar and sweeteners (Added),Total
Year,,,,,,,,
1970,508.895424,250.291001,71.242053,135.44114,409.802437,346.184520,332.539441,2054.396014
1971,516.361737,250.838785,73.058372,133.028418,403.733801,341.927818,335.319288,2054.268219
1972,515.380701,248.879915,68.71739,132.679686,399.035074,353.225707,339.185262,2057.103735
1973,486.202095,247.493426,71.771854,132.53015,410.792148,356.921131,340.257027,2045.967830
1974,502.622716,240.660385,72.5649,127.587458,406.397868,349.165931,328.810522,2027.809780
1975,494.111343,239.779865,75.998119,131.566708,415.616719,350.567626,317.701931,2025.342311
1976,516.431127,241.377387,77.274811,132.429937,429.822414,367.480149,334.509681,2099.325505
1977,512.02925,240.129874,77.642956,131.711948,417.396023,353.091598,342.454411,2074.456061
1978,506.419883,238.965042,77.612194,126.79488,427.027106,363.768219,338.537446,2079.124771


In [72]:
bucket_name = "s3-bucket-clean-usda"
obj_key = "loss-adjusted-food-availability-clean/calories_total.xls"
s3.df_to_s3(df, bucket_name, obj_key)